In [1]:
# IMPORT LIBRARIES
from bs4 import BeautifulSoup
from pyodbc import connect
import re
from deep_translator import GoogleTranslator
from random import randint

In [1]:
# FUNC translate_text
def translate_text(text):
    translator = GoogleTranslator(source='vi', target='en')
    try:
        return translator.translate(text)
    except Exception as e:
        return "Error: " + str(e)

In [2]:
# FUNC fetchAll, saveMany of SQL
def fetchAll(sql, parameters = None):
    db = connect('Driver={SQL Server};Server=.;Database=Hospital;UID;PWD')
    try:
        cursor = db.cursor()
        if parameters:
            cursor.execute(sql, parameters)
        else:
            cursor.execute(sql)
        return cursor.fetchall()
    except Exception as ex:
        print(ex)
        return False
    finally:
        cursor.close()
        db.close()
def saveMany(sql, arr):
    db = connect('Driver={SQL Server};Server=.;Database=Hospital;UID;PWD')
    try:
        cursor = db.cursor()
        cursor.executemany(sql, arr)
        ret = cursor.rowcount
        db.commit()
        return True
    except Exception as ex:
        print(ex)
        return False
    finally:
        cursor.close()
        db.close()

In [4]:
# FUNC append_dot
def append_dot(text):
    if re.search("[A-Za-z\s\u00C0-\u1EF9\d]+$", text):
        return text + '.'
    return text

In [6]:
# FUNC getIntroductions
def getIntrodutions(art):
    data = []
    with open(f'./data/{art[2]}.html') as f:
        soup = BeautifulSoup(f.readline(), 'html.parser')
        p = soup.select_one('p')
        try:
            while p.name == 'p' or p.name == None:
                text = p.text.strip()
                if text:
                    data.append((art[0], append_dot(text)))
                p = p.next
            return data
        except:
            print(art[2], p, data)

In [7]:
# Clear sql Paragraph, Item
def clearItems():
    try:
        db = connect('Driver={SQL Server};Server=.;Database=Hospital;UID=sa;PWD=123')
        cursor = db.cursor()
        sql = 'TRUNCATE TABLE Paragraph;TRUNCATE TABLE Item;'
        cursor.execute(sql)
        ret = cursor.rowcount
        db.commit()
        return True
    except Exception as ex:
        print(ex)
        return False
    finally:
        cursor.close()
        db.close()
clearItems()

In [ ]:
# Clear INTRODUCTION
def clearItems():
    try:
        db = connect('Driver={SQL Server};Server=.;Database=Hospital;UID=sa;PWD=123')
        cursor = db.cursor()
        sql = 'TRUNCATE TABLE Introduction;'
        cursor.execute(sql)
        ret = cursor.rowcount
        db.commit()
        return True
    except Exception as ex:
        print(ex)
        return False
    finally:
        cursor.close()
        db.close()
clearItems()

In [ ]:
# Get Introductions(articles) => data += getIntrodutions(art)
data = []
for art in fetchAll('SELECT * FROM Article'):
    data += getIntrodutions(art)
len(data)

In [ ]:
# INSERT data INTO Introduction(ArticleId, Content)
sql = 'INSERT INTO Introduction(ArticleId, Content) VALUES (?, ?)'
saveMany(sql, data)

In [ ]:
# FUNC getItemsAndParagraphs, extract_list
def extract_list(p): 
    arr = []
    for li in p.select('li'):
        text = li.text.strip()
        if(text):
            arr.append(append_dot(text))
    return ' '.join(arr)
def getItemsAndParagraphs(atr):
    items = []
    paragraphs = []
    with open(f'./data/{atr[2]}.html') as response:
        soup = BeautifulSoup(response, 'html.parser')
        for h2 in soup.select('div[id="ftwp-postcontent"] h2'):
            id = randint(0, 1073741824)
            items.append((atr[0], id, append_dot(h2.text.strip())))
            p = h2.next_sibling
            try:
                while p != None and (p.name in ('div', 'h3', 'h4', 'p', 'ul', 'ol', 'figure', 'blockquote') or p.name == None):
                    
                    if p.name in ('ul', 'ol'):
                        paragraphs.append((id, extract_list(p)))
                    else:
                        content = p.text.strip()
                        if content and len(content) > 3:
                            paragraphs.append((id, append_dot(content)))
                    p = p.next_sibling
            except Exception as ex:
                print(ex)
                print(atr[2], h2, p)
    return items, paragraphs

In [ ]:
# getItemsAndParagraphs
data_items = []
data_paragraphs = []
for art in fetchAll('SELECT * FROM Article'):
    items, paragraphs = getItemsAndParagraphs(art)
    data_items += items
    data_paragraphs += paragraphs
len(data_paragraphs)

In [ ]:
# Insert data into sql
sql = 'INSERT INTO Item (ArticleId, ItemId, ItemName) VALUES (?, ?, ?)'
saveMany(sql, data_items)
sql = 'INSERT INTO Paragraph (ItemId, Content) VALUES (?, ?)'
saveMany(sql, data_paragraphs)

In [ ]:
# FUNC Translate Introduction, save_batch
def batch(page, size = 1000):
    introduction = fetchAll('SELECT ArticleId, Content FROM Introduction ORDER BY IntroductionID OFFSET ? ROWS FETCH NEXT ? ROWS ONLY;', ((page-1)*size, size))
    translations = [translate_text(row[1]) for row in introduction]
    return introduction, translations
def save_batch(introduction, translations):
    trans = []
    for row, tran in zip(introduction, translations):
        trans.append((tran, row[0]))
    return saveMany('UPDATE Introduction SET ContentEn = ? WHERE ArticleId = ?', trans)

In [ ]:
# INSERT TRANSLATIONS INTO SQL DATA
for page in range (6):
    introduction, translations = batch(page,size = 1000)
    save_batch(introduction,translations)

In [ ]:
# English
sql = '''
SELECT Url, FieldName, ArticleName, STRING_AGG(ContentEn, ' ') AS Corpus 
FROM Introduction JOIN Article ON Introduction.ArticleId = Article.ArticleId
JOIN Field ON Field.FieldId = Article.FieldId
GROUP BY Introduction.ArticleId, ArticleName, Url, FieldName
UNION ALL
SELECT Url, FieldName, ItemName, STRING_AGG(ContentEn, ' ') AS Corpus FROM Paragraph
JOIN Item ON Paragraph.ItemId = Item.ItemId
JOIN Article ON Item.ArticleId = Article.ArticleId
JOIN Field ON Article.FieldId = Field.FieldId
GROUP BY Item.ArticleId, Url, Paragraph.ItemId, ItemName, FieldName ORDER BY Url;'''
data = fetchAll(sql)